# Basic DuckDB 🦆 for Spatial Analysis Setup

~ Dany Laksono 

The following is a basic steps for setting up duckdb and it's spatial extension, [PostGEESE](https://duckdb.org/2023/04/28/spatial.html). This notebook can be used as starting point for building poor man's data lake (or to be precise, data 'pond') for geospatial data science analytics using DuckDB's latest [Spatial Extension](https://github.com/duckdblabs/duckdb_spatial). 

Credit to:
- [Mark Litwintschik](https://tech.marksblogg.com/duckdb-geospatial-gis.html), one of the very first articles discussing about the wonderful of DuckDB to the geospatial community
- [Mark Forrest](https://www.youtube.com/watch?v=ljzpm3Mrw-I) for the inspiration.
- [Wei-Meng Lee](https://towardsdatascience.com/running-sql-queries-in-jupyter-notebook-using-jupysql-duckdb-and-mysql-3c53fbe40f8d) for the examples on JupySQL

## Preparation and Installation

In [1]:
%%writefile requirements.txt

duckdb
jupysql
duckdb-engine

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt --quiet

In [3]:
import duckdb
import pandas as pd

Setting up the JupySQL extension

In [4]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

/home/danylaksono/miniconda3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


The extension allows us to use `%sql` syntax in a jupyter lab cell for a single line SQL command, and `%%sql` syntax for multiline SQL command, as you'll see in the following cells

## Setup DuckDB and Install Spatial Extension

Initialize a DuckDB database in-memory 

In [5]:
%sql duckdb:///:memory:

Install the [spatial extension](https://duckdb.org/docs/extensions/spatial).

In [6]:
%%sql 
INSTALL spatial;
LOAD spatial;

""


DuckDB's [HTTPFS Extension](https://duckdb.org/docs/extensions/httpfs.html) allows us to load data into DuckDB _remotely_. Imagine the performance gained by taking advantage of S3's partial load for parquet data, while at the same time loading the data and processing them in DuckDB.

[This article](https://dagster.io/blog/duckdb-data-lake) by Dagster actually explores the possibility of building a working Data 'Pond' using DuckDB and  

In [7]:
%%sql
INSTALL httpfs;
LOAD httpfs;

""


Test a simple spatial operation

In [8]:
%%sql 
select st_point(0,0);

,"st_point(0, 0)"
0,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...


if there's anything wrong then congrats, it works!

## A Simple Use Case

In the following cells, we'll try and have some experiments on DuckDB. We'll read a GeoJSON data of London Trees (obtained from [here](https://data.london.gov.uk/dataset/local-authority-maintained-trees) and converted to GeoJSON for the demo). Just for a the sake of simplicity, we'll try to calculate the distance between each trees. There should be about

In [9]:
%%sql
create or replace table trees as
select *, ST_GeomFromWKB(wkb_geometry) as geom
from st_read('./london_trees.geojson')

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

,Count
0,817150


Show the dataset. Pay attention that SQL queries in DuckDB can be [simplified](https://duckdb.org/2022/05/04/friendlier-sql.html), so in the example below, we're omitting the 'SELECT *' and just go with 'FROM ... LIMIT' and so forth.

In [18]:
%%sql
from trees limit 10;

(duckdb.CatalogException) Catalog Error: Table with name trees does not exist!
Did you mean "temp.information_schema.tables"?
LINE 1: from trees limit 10;
             ^
[SQL: from trees limit 10;]
(Background on this error at: https://sqlalche.me/e/20/f405)


Here we're stepping up the game a little bit further. We can use DuckDB's [Native Geometry System](https://github.com/duckdblabs/duckdb_spatial#multi-tiered-geometry-type-system), which provides another performance gain for geospatial analysis

In [10]:
%%sql
alter table trees add column pointidx point_2d;

""


In [11]:
%%sql
update trees set pointidx = geom::point_2d;

,Count
0,817150


Now we calculate the distance. Notice the EXCLUDE? yeah that's DuckDB's [special SQL](https://duckdb.org/2022/05/04/friendlier-sql.html#select--exclude) syntax!

In this case, we'll use the GEOM column, which follows the Simple Features geometry model. This geom can be used for interoperability, since it will have the same structure with other standardised geometry (such as Geopandas or PostGIS).

Notice that this operation is quite fast: after all, we're calculating ~800k distances!

In [14]:
%%time
%%sql
select * exclude (objectid, gla_tree_n, tree_name, age, spread_m, wkb_geometry), 
st_distance(geom, st_point(51.65345594544537, -0.19995380867530435)) as distance 
from trees

CPU times: user 5.36 s, sys: 432 ms, total: 5.8 s
Wall time: 6.23 s


,fid,taxon_name,common_nam,age_group,height_m,canopy_spr,diameter_a,dbh_group,longitude,latitude,condition,load_date,updated,geom,pointidx,distance
0,1,Abies grandis,Grand fir,Early mature (16-30),10 to 15m,00 to 05m,NaN,21 to 40cm,-0.291147,51.361893,Reasonable,20210318,20210715,"b""\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...","{'x': -0.29114702, 'y': 51.36189321}",73.190613
1,2,Abies grandis,Grand fir,Early mature (16-30),10 to 15m,00 to 05m,NaN,21 to 40cm,-0.291122,51.361914,Reasonable,20210318,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': -0.29112242, 'y': 51.36191363}",73.190610
2,3,Abies grandis,Grand fir,Early mature (16-30),05 to 10m,05 to 10m,NaN,21 to 40cm,-0.290943,51.387016,Reasonable,20210318,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': -0.29094338, 'y': 51.38701648}",73.208170
3,4,Abies grandis,Grand fir,Mature (31-80),10 to 15m,05 to 10m,NaN,41 to 70cm,-0.288572,51.387405,Reasonable,20210318,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': -0.28857187, 'y': 51.38740453}",73.206761
4,5,Abies grandis,Grand fir,Mature (31-80),10 to 15m,05 to 10m,NaN,41 to 70cm,-0.285025,51.388872,Reasonable,20210318,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': -0.28502456, 'y': 51.38887244}",73.205278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817145,817146,Prunus avium,Cherry,None,None,None,NaN,None,0.013561,51.514825,None,20180214,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': 0.01356119, 'y': 51.51482525}",73.082810
817146,817147,Carpinus betulus fastigiata,Hornbeam,None,None,None,NaN,None,0.039496,51.538224,None,20180214,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': 0.03949611, 'y': 51.53822397}",73.081050
817147,817148,Acer pseudoplatanus,Sycamore,None,None,None,NaN,None,0.053449,51.510719,None,20180214,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': 0.05344862, 'y': 51.51071912}",73.051724
817148,817149,Robinia pseudoacacia,False acacia,None,None,None,NaN,None,0.053493,51.510352,None,20180214,20210715,b'\x00\x00\x18\x00\x00\x00\x00\x00\x00\x00\x00...,"{'x': 0.05349266, 'y': 51.51035235000001}",73.051434


Now let's try to calculate them again, but this time we'll use DuckDB's Native Geometry, which we've built before.

In [15]:
%%time
%%sql
select common_nam, st_distance(pointidx, st_point(51.65345594544537, -0.19995380867530435)::point_2d) as distance from trees

CPU times: user 1.05 s, sys: 39.4 ms, total: 1.09 s
Wall time: 1.33 s


,common_nam,distance
0,Grand fir,73.190613
1,Grand fir,73.190610
2,Grand fir,73.208170
3,Grand fir,73.206761
4,Grand fir,73.205278
...,...,...
817145,Cherry,73.082810
817146,Hornbeam,73.081050
817147,Sycamore,73.051724
817148,False acacia,73.051434


And that's ever faster!

So I rest my case: Q.E.D.

## What's next?

At the moment, the spatial analysis functions in DuckDB is still in it's infancy, but we can expect them to grow overtime. Even now, we can explore some of the functionalities, such as reading data directly from [PostgreSQL](https://duckdb.org/docs/extensions/postgres_scanner). There are also some useful extensions, such as [Isaac Brodsky's H3](https://github.com/isaacbrodsky/h3-duckdb) which could really be a game changer for processing large geospatial dataset. Have fun!